# Лабораторная работа 5: Обучение и тестирование модели

Для начала выполним импорт библиотек

In [ ]:
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import zipfile
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Any

Читаем пути из аннотаций формата csv

In [ ]:
cat_dframe = pd.read_csv(cat_csv_path, delimiter=",", names=["Absolute path", "Relative path", "Class"])
    dog_dframe = pd.read_csv(dog_csv_path, delimiter=",", names=["Absolute path", "Relative path", "Class"])

    cat_images = cat_dframe["Absolute path"].tolist()
    dog_images = dog_dframe["Absolute path"].tolist()

    return cat_images, dog_images

img in index 0: The absolute path

img in index 1: C:/Users/User/Desktop/testing/dataset\cat\cat.4001.jpg

img in index 2: C:/Users/User/Desktop/testing/dataset\cat\cat.4002.jpg

img in index 0: The absolute path

img in index 1: C:/Users/User/Desktop/testing/dataset\dog\dog.4001.jpg

img in index 2: C:/Users/User/Desktop/testing/dataset\dog\dog.4002.jpg

Разделение данных на тренировочные, тестовые и валидационные в пропорциях: 80 на 10 на 10, соответственно

In [ ]:
cat_train_data, cat_test_val_data = train_test_split(cat_images, test_size=(test_size + val_size), random_state=42)
    cat_test_data, cat_valid_data = train_test_split(cat_test_val_data, test_size=(val_size / (test_size + val_size)), random_state=42)

    dog_train_data, dog_test_val_data = train_test_split(dog_images, test_size=(test_size + val_size), random_state=42)
    dog_test_data, dog_valid_data = train_test_split(dog_test_val_data, test_size=(val_size / (test_size + val_size)), random_state=42)

    train_data = cat_train_data + dog_train_data
    test_data = cat_test_data + dog_test_data
    valid_data = cat_valid_data + dog_valid_data

    return train_data, test_data, valid_data

1619
202
204

Проверка изображений

In [ ]:
random_idx = np.random.randint(1,len(img_list),size=10)

fig = plt.figure()
i=1
for idx in random_idx:
    ax = fig.add_subplot(2,5,i)
    img = Image.open(img_list[idx])
    plt.imshow(img)
    i+=1
    plt.axis('off')


plt.show()

![title](C:\Users\User\Desktop\testing\ipynb\fig1.png)
![title](C:/Users/User/Desktop/testing/ipynb/fig2.png)
![Alt text](C:/Users/User/Desktop/testing/ipynb/fig1.png)
![Alt text](C:/Users/User/Desktop/testing/ipynb/fig2.png)